# Core Functions
> Summary description here. 

In [1]:
#hide
# default_exp core
from nbdev.showdoc import *

In [15]:
# export

import numpy as np
import cv2
import matplotlib.pyplot as plt
import PIL
import os

In [3]:
# export
def progress_bar(i,n):
    print(':', end='')
    if i % n == 0:
        print(' ', i)
        

In [4]:
# export
import csv
def resize_file(fn, scale=1):
    img = PIL.Image.open(fn)
    img_w, img_h = img.size
    img_w, img_h = int(img_w*scale), int(img_h*scale)
    return img.resize((img_w, img_h))
        

In [6]:
# export
def show_img(im, figsize=None, ax=None, alpha=None, label=None, title=None):
    if not ax: fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
    ax.set_axis_off()
    if label: ax.legend()
    if title: ax.set_title(title)
    return ax

#export
def in_ipython():
    "Check if the code is running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))
    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False

IN_IPYTHON = in_ipython()

#export
def in_colab():
    "Check if the code is running in Google Colaboratory"
    try:
        from google import colab
        return True
    except: return False

IN_COLAB = in_colab()

#export
def in_notebook():
    "Check if the code is running in a jupyter notebook"
    if in_colab(): return True
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell': return True   # Jupyter notebook, Spyder or qtconsole
        elif shell == 'TerminalInteractiveShell': return False  # Terminal running IPython
        else: return False  # Other type (?)
    except NameError: return False      # Probably standard Python interpreter

IN_NOTEBOOK = in_notebook()

#export
from pynvml import *
def gpu_memory():
    'Print memory total, free and used'
    nvmlInit()
    h = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(h)
    print('*** GPU memory ***')
    print(f'total    : {info.total}')
    print(f'free     : {info.free}')
    print(f'used     : {info.used}')

In [8]:
# export
def colormap_segmentation_labels(N=256):
    'create a colour map for  segmentation labels'
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    cmap = np.zeros((N, 3))
    for i in range(0, N):
        id = i
        r, g, b = 0, 0, 0
        for j in range(0, 8):
            r = np.bitwise_or(r, (bitget(id, 0) << 7 - j))
            g = np.bitwise_or(g, (bitget(id, 1) << 7 - j))
            b = np.bitwise_or(b, (bitget(id, 2) << 7 - j))
            id = (id >> 3)
        cmap[i, 0] = r
        cmap[i, 1] = g
        cmap[i, 2] = b
    # cmap = cmap.astype(np.float32) / 255
    cmap = cmap.astype(np.uint8).flatten()
    return cmap

# export
def save_png_p(img:np.ndarray, fn, cmap:str='hot', num_colors=256):
    '''
    Save an image as an 8 bit png file with a color map
    - img: 
    - fn: filename to save
    - cmap: colormap ( use matplotlib cmaps )
    - num_colors:  less than 256, for label images set `num_colors` to the number of colors in the `cmap` colormap 
    '''

    assert num_colors > 0 and num_colors <= 256, 'num_colors must be in range 0 to 256'
    cm = plt.get_cmap(cmap)
    palette = np.ones((256,3),dtype=np.uint8)*128
    pal = (cm(np.arange(num_colors))*255).astype(np.uint8)[:,:3]
    palette[:num_colors,:] = pal
    img = PIL.Image.fromarray((img*255).astype(np.uint8), mode='P')
    img.putpalette(palette)
    img.save(fn)

      

> 
>For an exapmple of available colour maps see
<img alt="Example colour maps" width="800" caption="Example colour maps" src="images/colormaps_001.png">

<img alt="Example colour maps" width="800" caption="Example colour maps" src="images/colormaps_002.png">

> Use a Quantative color map for label images and set number to the number of colors in the colormap 
`save_png_p(img, 'file.png', cmap='tab10', num_colors=10)`

><img alt="Quantative color maps" width="800" caption="Quantative color maps" src="images/colormaps_reference_04.png">

In [16]:
img = np.random.rand(10,10)
img = cv2.resize(img, dsize=(200, 200), interpolation=cv2.INTER_NEAREST)
save_png_p(np.resize(img,(200,200)), 'images/save_png_p_test.png', cmap='hot')

<img alt="images/save_png_p_test.png" width="400" caption="images/save_png_p_test.png" src="images/save_png_p_test.png">